# Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
import schwabdev
from scipy import stats
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
%matplotlib inline

# Momentum Investing
- the strategy in investing in the stocks that increased in price the most

# Collecting S&P 500 constituents

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_constituents = pd.read_html(sp_url, header=0)[0]

In [3]:
sp500_constituents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [4]:
sp500_constituents = sp500_constituents['Symbol'].to_list()

### changing format to match Schwab's

In [5]:
sp500_constituents = [x.replace('.', '/') for x in sp500_constituents]

# Setting up Schwab API

In [6]:
from key import api_key, api_secret

In [7]:
client = schwabdev.Client(app_key=api_key, app_secret=api_secret, capture_callback=False)

# Creating Datafram
- Basing this strategy in a 6 month change

In [8]:
my_columns = ['Ticker', 'Price (6 months)', 'Price (now)', 'Net Change', 'Number of Shares to Buy']

In [9]:
today = datetime.today()
six_months = today - relativedelta(months = 6)

In [10]:
entries = []
for symbol in sp500_constituents:
    price_history = client.price_history(symbol, periodType='month', period = 6, startDate=six_months, endDate=today).json()
    price_now = price_history['candles'][-1]['close']
    price_6_months = price_history['candles'][0]['close']
    net_change = round(((price_now - price_6_months)/ price_6_months) * 100, 2)
    entries.append(
        [
            symbol,
            price_6_months,
            price_now,
            net_change,
            'N/a'
        ]
    )

In [11]:
momentum_df = pd.DataFrame(entries, columns=my_columns)

### Sorting by biggest increases

In [12]:
momentum_df.sort_values(by='Net Change', ascending=False, inplace=True)

In [13]:
momentum_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
360,PLTR,41.92,94.07,124.40,N/a
333,NFLX,756.10,1050.11,38.89,N/a
468,VRSN,176.68,245.12,38.74,N/a
434,TPR,46.97,64.43,37.17,N/a
171,EQT,35.62,48.71,36.75,N/a


# Creating Excel Output with all data

In [14]:
momentum_df_copy = momentum_df.copy()

In [15]:
momentum_df_copy = momentum_df_copy.drop('Number of Shares to Buy', axis=1)
momentum_df_copy.head()

,Ticker,Price (6 months),Price (now),Net Change
360,PLTR,41.92,94.07,124.40
333,NFLX,756.10,1050.11,38.89
468,VRSN,176.68,245.12,38.74
434,TPR,46.97,64.43,37.17
171,EQT,35.62,48.71,36.75


In [16]:
writer = pd.ExcelWriter('total_data.xlsx', engine='xlsxwriter')

momentum_df_copy.to_excel(writer, sheet_name='Movers', index=False)

### Formatting

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [18]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [19]:
percent_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying Formats

In [20]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format]
}

In [21]:
for column in column_formats.keys():
    writer.sheets['Movers'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Movers'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

In [22]:
writer.close()

# Keeping Top 50 Movers

In [23]:
final_df = momentum_df.copy()

In [24]:
final_df = final_df.head(50)
final_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 360 to 60
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   50 non-null     object 
 1   Price (6 months)         50 non-null     float64
 2   Price (now)              50 non-null     float64
 3   Net Change               50 non-null     float64
 4   Number of Shares to Buy  50 non-null     object 
dtypes: float64(3), object(2)
memory usage: 2.3+ KB


In [25]:
final_df.reset_index(inplace=True)
final_df.drop('index', axis = 1, inplace=True)
final_df

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,41.920,94.0700,124.40,N/a
1,NFLX,756.100,1050.1100,38.89,N/a
2,VRSN,176.680,245.1200,38.74,N/a
3,TPR,46.970,64.4300,37.17,N/a
4,EQT,35.620,48.7100,36.75,N/a
5,MCK,526.340,688.5000,30.81,N/a
6,AXON,425.840,551.7400,29.57,N/a
7,CNP,28.580,37.0250,29.55,N/a
8,KR,56.570,73.1400,29.29,N/a
9,TTWO,163.780,209.8850,28.15,N/a


# Calculating the Number of Shares to Buy

In [26]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of the porfolio: ')

    try:
        float(portfolio_size)
    except ValueError:
        print('Invalid input! \n Please try again.')
        portfolio_size = input('Enter the size of the porfolio: ')


In [27]:
portfolio_input()

In [28]:
print(portfolio_size)

100000


In [29]:
position_size = float(portfolio_size) / len(final_df.index)
position_size

2000.0

In [30]:
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Price (now)'])

In [31]:
final_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,41.92,94.07,124.40,21
1,NFLX,756.10,1050.11,38.89,1
2,VRSN,176.68,245.12,38.74,8
3,TPR,46.97,64.43,37.17,31
4,EQT,35.62,48.71,36.75,41


# Exporting to Excel

In [32]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

final_df.to_excel(writer, sheet_name='Recommended Trades', index=False)

In [33]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [34]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [35]:
percent_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [36]:
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [37]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format],
    'E': ['Number of Shares to Buy', integer_format]
}

In [38]:
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [39]:
writer.close()

# More Realistic Momentum Strategy

Want to differentiate from high quality and low quality
- high quality - have slow and steady outperformance over long period of time
- low quality - might be from one surge of increase 

We want high quality momentum stocks because it shows repeated success rather than a surge in increase due to one event.

In [40]:
hqm_columns = [
    'Ticker',
    'Price',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]

hqm_df = pd.DataFrame(columns=hqm_columns)
hqm_df

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile


## Making API Calls

In [41]:
today = datetime.today()
one_year = today - relativedelta(years = 1, months = 1)

In [42]:
print(today)
print(one_year)

2025-04-22 10:27:53.150694
2024-03-22 10:27:53.150694


In [43]:
symbol = 'AAPL'
price_history = client.price_history(symbol, periodType='year', frequencyType='monthly', frequency=1, startDate=one_year, endDate=today).json()

In [44]:
print(price_history['candles'][6]['datetime'])

1727758800000


In [45]:
entries = []
for symbol in sp500_constituents:
    price_history = client.price_history(symbol, periodType='month', frequency=1, startDate=one_year, endDate=today).json()
    price_now = price_history['candles'][-1]['close']
    price_12_months = price_history['candles'][0]['close']
    price_6_months = price_history['candles'][6]['close']
    price_3_months = price_history['candles'][9]['close']
    price_1_month = price_history['candles'][-2]['close']
    net_change_12= round(((price_now - price_12_months)/ price_12_months) * 100, 2)
    net_change_6 = round(((price_now - price_6_months)/ price_6_months) * 100, 2)
    net_change_3 = round(((price_now - price_3_months)/ price_3_months) * 100, 2)
    net_change_1 = round(((price_now - price_1_month)/ price_1_month) * 100, 2)
    entries.append(
        [
            symbol,
            price_now,
            net_change_12,
            'N/a',
            net_change_6,
            'N/a',
            net_change_3,
            'N/a',
            net_change_1,
            'N/a'
        ]
    )

In [46]:
hqm_df = pd.DataFrame(entries, columns=hqm_columns)

In [47]:
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,135.535,52.82,N/a,37.00,N/a,35.35,N/a,4.09,N/a
1,AOS,62.420,-30.23,N/a,-27.74,N/a,-25.37,N/a,-1.14,N/a
2,ABT,131.420,15.63,N/a,25.47,N/a,28.60,N/a,0.34,N/a
3,ABBV,172.155,-5.46,N/a,7.09,N/a,6.77,N/a,-0.48,N/a
4,ACN,282.150,-18.60,N/a,-7.89,N/a,-0.05,N/a,-0.07,N/a


## Calculating Momentum Percentiles 

In [48]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for period in time_periods:
        hqm_df.loc[row, f'{period} Return Percentile'] = stats.percentileofscore(
            hqm_df[f'{period} Price Return'],
            hqm_df.loc[row, f'{period} Price Return']
        )

In [49]:
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,135.535,52.82,96.620278,37.00,93.240557,35.35,93.439364,4.09,98.011928
1,AOS,62.420,-30.23,14.910537,-27.74,13.717694,-25.37,15.506958,-1.14,34.194831
2,ABT,131.420,15.63,77.534791,25.47,87.276342,28.60,88.866799,0.34,74.055666
3,ABBV,172.155,-5.46,46.918489,7.09,66.998012,6.77,64.811133,-0.48,51.192843
4,ACN,282.150,-18.60,28.827038,-7.89,41.749503,-0.05,50.795229,-0.07,63.220676


## Calculating HQM Score
- HQM Score - mean of the four percentiles scores

In [50]:
from statistics import mean

In [51]:
for i in range(len(hqm_df)):
    momentum_percentiles = []
    for period in time_periods:
        momentum_percentiles.append(hqm_df.loc[i, f'{period} Return Percentile'])
    hqm_df.loc[i, 'HQM Score'] = mean(momentum_percentiles)

In [52]:
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,135.535,52.82,96.620278,37.00,93.240557,35.35,93.439364,4.09,98.011928,95.328032
1,AOS,62.420,-30.23,14.910537,-27.74,13.717694,-25.37,15.506958,-1.14,34.194831,19.582505
2,ABT,131.420,15.63,77.534791,25.47,87.276342,28.60,88.866799,0.34,74.055666,81.933400
3,ABBV,172.155,-5.46,46.918489,7.09,66.998012,6.77,64.811133,-0.48,51.192843,57.480119
4,ACN,282.150,-18.60,28.827038,-7.89,41.749503,-0.05,50.795229,-0.07,63.220676,46.148111


### Getting top performing stocks

In [53]:
hqm_df.sort_values(by='HQM Score', ascending=False, inplace=True)
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
333,NFLX,1049.510,72.81,98.807157,71.81,99.204771,63.57,99.005964,7.86,99.801193,99.204771
481,WMT,94.640,57.29,98.011928,56.48,98.011928,43.92,96.222664,1.52,90.755467,95.750497
0,MMM,135.535,52.82,96.620278,37.00,93.240557,35.35,93.439364,4.09,98.011928,95.328032
216,GILD,105.545,44.09,95.427435,60.01,98.409543,64.22,99.204771,0.96,85.785288,94.706759
146,DFS,170.530,30.09,89.662028,38.19,94.035785,39.03,94.4334,6.83,99.403579,94.383698


### exporting to excel

In [71]:
workbook = xlsxwriter.Workbook('total_data.xlsx')
worksheet = workbook.add_worksheet('Better Movers')

writer = pd.ExcelWriter('total_data.xlsx', engine='xlsxwriter')

hqm_df.to_excel(writer, sheet_name='Better Movers', index=False)

In [73]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [74]:
hqm_column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['One-Year Price Return', percent_format],
    'D': ['One-Year Return Percentile', percent_format],
    'E': ['Six-Month Price Return', percent_format],
    'F': ['Six-Month Return Percentile', percent_format],
    'G': ['Three-Month Price Return', percent_format],
    'H': ['Three-Month Return Percentile', percent_format],
    'I': ['One-Month Price Return', percent_format],
    'J': ['One-Month Return Percentile', percent_format],
    'K': ['HQM Score', percent_format]
}

In [75]:
for column in column_formats.keys():
    writer.sheets['Better Movers'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Better Movers'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [76]:
writer.close()